## Database statistics for SCI Data publication - Will give the contents of Table 1

In [1]:
from SlideRunner.dataAccess.database import Database
import SlideRunner.dataAccess.database
import os
import numpy as np
import openslide
import cv2

In [2]:

test_slide_filenames = ['f26e9fcef24609b988be.svs',
'dd4246ab756f6479c841.svs',
'f3741e764d39ccc4d114.svs',
'c91a842257ed2add5134.svs',
'8c9f9618fcaca747b7c3.svs',
'add0a9bbc53d1d9bac4c.svs',
'c86cd41f96331adf3856.svs',
'be10fa37ad6e88e1f406.svs',
'96274538c93980aad8d6.svs',
'552c51bfb88fd3e65ffe.svs',
'1018715d369dd0df2fc0.svs']



## Query all databases for the respective classes. Joint query to have all in the same order.

In [3]:
DB = Database().open('MITOS_WSI_CCMCT_ODAEL.sqlite')
DB.execute('ATTACH `MITOS_WSI_CCMCT_MEL.sqlite` as MEL;')
DB.execute('ATTACH `MITOS_WSI_CCMCT_HEAEL.sqlite` as HEAEL;')

ODAEL = DB.execute("""
SELECT * FROM (
SELECT filename, MitoticFigures, MitoticFigureLookalikes, Granulocytes, TumorCells FROM 
(SELECT COUNT(*) as MitoticFigureLookalikes, slide from Annotations where agreedClass==7 group by slide) as cls7
LEFT JOIN (SELECT COUNT(*) as MitoticFigures, slide from Annotations where agreedClass==2 group by slide) as cls2 on cls2.slide == cls7.slide
LEFT JOIN (SELECT COUNT(*) as Granulocytes, slide from Annotations where agreedClass==1 group by slide) as cls1 on cls1.slide == cls7.slide
LEFT JOIN (SELECT COUNT(*) as TumorCells, slide from Annotations where agreedClass==3 group by slide) as cls3 on cls3.slide == cls7.slide
LEFT JOIN Slides on Slides.uid == cls2.slide order by mitoticFigures) as ODAEL
LEFT JOIN 
(
SELECT filename, MitoticFigures, 0 as MitoticFigureLookalikes, Granulocytes, TumorCells FROM (
SELECT COUNT(*) as MitoticFigures, slide from MEL.Annotations where agreedClass==2 group by slide) as cls2
LEFT JOIN (SELECT COUNT(*) as Granulocytes, slide from MEL.Annotations where agreedClass==1 group by slide) as cls1 on cls1.slide == cls2.slide
LEFT JOIN (SELECT COUNT(*) as TumorCells, slide from MEL.Annotations where agreedClass==3 group by slide) as cls3 on cls3.slide == cls2.slide
LEFT JOIN MEL.Slides on Slides.uid == cls2.slide order by mitoticFigures
) as MEL on MEL.filename == ODAEL.filename
LEFT JOIN 
(
SELECT filename, MitoticFigures, MitoticFigureLookalikes, Granulocytes, TumorCells FROM 
(SELECT COUNT(*) as MitoticFigureLookalikes, slide from HEAEL.Annotations where agreedClass==7 group by slide) as cls7
LEFT JOIN (SELECT COUNT(*) as MitoticFigures, slide from HEAEL.Annotations where agreedClass==2 group by slide) as cls2 on cls2.slide == cls7.slide
LEFT JOIN (SELECT COUNT(*) as Granulocytes, slide from HEAEL.Annotations where agreedClass==1 group by slide) as cls1 on cls1.slide == cls2.slide
LEFT JOIN (SELECT COUNT(*) as TumorCells, slide from HEAEL.Annotations where agreedClass==3 group by slide) as cls3 on cls3.slide == cls2.slide
LEFT JOIN MEL.Slides on Slides.uid == cls2.slide order by mitoticFigures
) as HEAEL on HEAEL.filename == ODAEL.filename


""").fetchall()

Output everything in LateX table format

In [4]:


dbTumor = Database()

dbTumor.open('./MITOS_WSI_CCMCT_Tumorzone.sqlite')
totalarea = 0
totalmit = 0

for (filename, mit,nonmit, gran, tum, _, mitA, nonmitA, granA, tumA,_, mitH,nonmitH, granH, tumH) in ODAEL:

    dbTSlide = dbTumor.findSlideWithFilename(os.path.basename(filename), slidepath='')
    dbTumor.loadIntoMemory(dbTSlide)
    
    slide = openslide.open_slide('../WSI/'+filename)


    positiveId = dbTumor.findClassidOfClass('Tumorzone')[0][0]
    import matplotlib.path as p

    micronsPerPixel = slide.properties[openslide.PROPERTY_NAME_MPP_X]
    mpp_scaled = float(slide.level_downsamples[-1]) * float(micronsPerPixel)  

    slideActiveMap = np.zeros((slide.level_dimensions[-1][::-1]))
    ds = slide.level_downsamples[-1]
    for anno in dbTumor.annotations.keys():
        if (dbTumor.annotations[anno].agreedLabel()==positiveId) and (type(dbTumor.annotations[anno]) == type(dbTumor.annotations[anno]) == SlideRunner.dataAccess.annotations.polygonAnnotation):
            co = dbTumor.annotations[anno].coordinates
            p3 = np.int32([np.asarray(co)/ds])
            cv2.fillPoly(slideActiveMap, p3,color=[255] )

    for anno in dbTumor.annotations.keys():
        if not (dbTumor.annotations[anno].agreedLabel()==positiveId) and (type(dbTumor.annotations[anno]) == type(dbTumor.annotations[anno]) == SlideRunner.dataAccess.annotations.polygonAnnotation):
            co = dbTumor.annotations[anno].coordinates
            p3 = np.int32([np.asarray(co)/ds])
            cv2.fillPoly(slideActiveMap, p3,color=[0] )
    mm2 = np.sum(slideActiveMap>0)*mpp_scaled*mpp_scaled/1E6
    totalarea += mm2
    totalmit += mit
                
    print('%s & %.2f\,$\\mathrm{mm}^2$ & %d / %d / %d & %d / %d / %d & %d / %d / %d & %d / %d / %d & %s \\\\ ' % 
          (filename.replace('_','\\_'), mm2, mit,mitH,mitA,nonmit,nonmitH,nonmitA, gran,granH, granA, tum, tumH, tumA, 'test' if filename in test_slide_filenames else 'train'))

2f2591b840e83a4b4358.svs & 144.79\,$\mathrm{mm}^2$ & 3 / 1 / 1 & 48 / 2 / 0 & 2213 / 2213 / 2213 & 1149 / 1113 / 1113 & train \\ 
ce949341ba99845813ac.svs & 13.94\,$\mathrm{mm}^2$ & 4 / 1 / 1 & 30 / 2 / 0 & 35 / 35 / 35 & 1200 / 1197 / 1197 & train \\ 
91a8e57ea1f9cb0aeb63.svs & 25.24\,$\mathrm{mm}^2$ & 6 / 2 / 2 & 16 / 3 / 0 & 573 / 573 / 573 & 1916 / 1903 / 1903 & train \\ 
9374efe6ac06388cc877.svs & 35.63\,$\mathrm{mm}^2$ & 7 / 6 / 6 & 17 / 4 / 0 & 1531 / 1531 / 1531 & 1567 / 1534 / 1534 & train \\ 
0e56fd11a762be0983f0.svs & 25.63\,$\mathrm{mm}^2$ & 8 / 4 / 4 & 262 / 17 / 0 & 239 / 239 / 239 & 1620 / 1089 / 1089 & train \\ 
dd6dd0d54b81ebc59c77.svs & 62.25\,$\mathrm{mm}^2$ & 11 / 5 / 5 & 57 / 15 / 0 & 1230 / 1230 / 1230 & 1830 / 1733 / 1733 & train \\ 
be10fa37ad6e88e1f406.svs & 14.87\,$\mathrm{mm}^2$ & 12 / 3 / 3 & 55 / 2 / 0 & 124 / 124 / 124 & 1354 / 1351 / 1351 & test \\ 
2e611073cff18d503cea.svs & 81.64\,$\mathrm{mm}^2$ & 18 / 11 / 11 & 137 / 2 / 0 & 2556 / 2556 / 2556 & 1136 

## And finally the question: What is the total area?

In [5]:
print('Total area: ',totalarea,'mm^2, mitotic figures in ODAEL data set',totalmit)

Total area:  4842.062112944357 mm^2, mitotic figures in ODAEL data set 44880
